In [20]:
!pip install -qU langchain-openai
!pip install -qU langchain-core
!pip install -qU tiktoken
!pip install -qU langchain
!pip install -qU langchain-community
!pip install -qU uvicorn
!pip install -qU pymupdf
!pip install -qU qdrant-client
!pip install -qU python-dotenv
!pip install -qU fastapi
!pip install -qU pypdf
!pip install -qU fastapi
!pip install -qU python-multipart
!pip install -qU python-jobspy


In [21]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [47]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"RefineMyResume - Agents - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

In [29]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough
from utils import *
import os
import getpass
from langchain.globals import set_debug
import chainlit as cl 
from langchain_openai import ChatOpenAI, OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
import csv
import json
import pandas as pd
from langchain_core.messages import HumanMessage, ToolMessage
from typing import Any, Callable, List, Optional, TypedDict, Union, Annotated

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph
import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import functools

In [30]:
llm_name: str = "gpt-3.5-turbo-0125"
evalutor_llm_name: str = "gpt-4-turbo"
embedding_model: str = "text-embedding-3-small"


In [31]:
openai_chat_model = ChatOpenAI(temperature=0.1, top_p=0.0001, model="gpt-3.5-turbo")
openai_evalutor_model = ChatOpenAI(temperature=0.1, top_p=0.0001, model=evalutor_llm_name)
enc = tiktoken.encoding_for_model(llm_name)
pages = PyPDFLoader("data/resume.pdf").load_and_split()
embedding_model = OpenAIEmbeddings(model=embedding_model)


/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(
/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


In [32]:
def read_parse_resume():
    PROMPT_4="""
        You are given resume : ```{resume}```

        then based on the given resume extract information about the person

        {format_instructions}

        """
    template = PROMPT_4
    parser = PydanticOutputParser(pydantic_object=OutputFormat)
    prompt_template_name = PromptTemplate(
        input_variables=["resume"],
        template=template,
        partial_variables={"format_instructions": parser.get_format_instructions()},
        )

    name_chain = LLMChain(llm=openai_chat_model, prompt=prompt_template_name)
    response = name_chain(inputs={"resume": pages})
    resume_json=parser.parse(response["text"]).json()
    return resume_json
    
def scan_for_jobs():
    jobs = scrape_jobs(
        site_name=["indeed", "linkedin", "zip_recruiter", "glassdoor"],
        search_term="software engineer",
        location="San Francisco, CA",
        results_wanted=20,
        hours_old=72, # (only Linkedin/Indeed is hour specific, others round up to days old)
        country_indeed='USA',  # only needed for indeed / glassdoor
        # linkedin_fetch_description=True # get full description and direct job url for linkedin (slower)
    )
    print(f"Found {len(jobs)} jobs")
    jobs.to_csv("data/jobs.csv", quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False) # to_xlsx
    return jobs
    
def write_resume_to_file(resume_json, fileName:str='data/resume.json'): 
    f=open(fileName, 'w')
    f.write(resume_json)
    f.close()
        
def get_resume(path):
    if os.path.isfile(path):
        with open(path) as json_file:
            print('Loading resume.json from the disk')
            resume_json= json.load(json_file)
    else:
        print('Reading pdf resume and Loading')
        resume_json = read_parse_resume()
        write_resume_to_file(resume_json)
    return resume_json

def get_jobs(path):
    if os.path.isfile(path):
        print('Loading jobs.csv from the disk')
        csvFile = df=pd.read_csv(path)
    else:
        print('Scanning web for jobs and Loading')
        csvFile= scan_for_jobs()
    return csvFile

In [33]:
resume = get_resume("data/resume.json")
summary = resume["About_Me"]["Summary"]
keywords = resume["Skills"]["Technical_Skills"]


Loading resume.json from the disk


In [34]:
summary

'Over the past 20 years I have continuously challenged myself with new languages, frameworks and methodologies. A full stack web developer and have recently been working more with Angular, React/react-native, AI/ML, Android, Blockchain/Hyperledger Fabric and Composer. An expert in front-end client-side technologies, backend server-side technologies and the database technologies.'

In [35]:
keywords

['Angular',
 'TypeScript',
 'Python (Numpy, scikit-learn, keras, PyTorch, Pandas, TensorFlow)',
 'RxJS',
 'React/React-Native',
 'Redux/Flux',
 'Hyperledger Fabric/Composer',
 'Blockchain',
 'Node.js',
 'JavaScript',
 'IOT',
 'Java',
 'Android',
 'AWS',
 'GCP',
 'Application Architecture',
 'CSS3',
 'REST',
 'SQL',
 'NOSQL (MongoDB, CouchDB)']

In [36]:
jobs_list =get_jobs("data/jobs.csv")
firstJob = jobs_list.iloc[0]
job_description = firstJob.description

Loading jobs.csv from the disk


In [66]:
@tool
def missing_keywords():
    """
        Given a set of keywords and a job description, finds all the missing keywords in the job description.
    """
    SYSTEM_PROMPT = "You are an expert in reading resume's and job description"
    USER_PROMPT = """
            You are here to find the missing keywords or skills from a Resume.             
            
            You are given a job description. Here is the job description:
            {job_description}
            
            You are also given the list of user skillset :
            {keywords}
            
            List out all the skills which are in the job description but missing from the user skillset in the order of importance.
        """
    ASSISTANT_PROMPT= "For example, GraphSQL is in the job description but missing from the resume."
    chat_prompt = ChatPromptTemplate.from_messages(
            messages = [
                ("system", SYSTEM_PROMPT),
                ("assistant", ASSISTANT_PROMPT),
                ("user", USER_PROMPT)
        ])
    chain = chat_prompt | openai_chat_model ## add pydantic parser, 
    return chain.invoke({"job_description": job_description, "keywords": keywords})

In [67]:
@tool
def inconsistencies_in_resume():
    """
        Finds conflicting information, date inconsistencies in the given resume        
    """
    SYSTEM_PROMPT = "You are an expert in reading resume's"
    USER_PROMPT = """
        You are given resume ```{resume}```. 

        List out all the conflicting information, date inconsistencies.
    """
    chat_prompt = ChatPromptTemplate.from_messages(
        messages = [
            ("system", SYSTEM_PROMPT),
            ("user", USER_PROMPT)
    ])
    chain = chat_prompt | openai_chat_model ## add pydantic parser, 
    return chain.invoke({resume: resume})

In [68]:
tools = [missing_keywords, inconsistencies_in_resume]
llm_with_tools = openai_chat_model.bind_tools(tools)

In [70]:
from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

In [71]:
from langchain.globals import set_debug

# set_debug(True)

# llm_with_tools.invoke("Find inconsistencies in resume")
# llm_with_tools.invoke("Find the missing keywords in the resume")
tool_executor.invoke("Find the missing keywords in the resume")

AttributeError: 'str' object has no attribute 'tool'